# Importing Libraries

In [19]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
from torchvision.utils import save_image
import matplotlib.pyplot as plt

# Loading the Model

In [20]:
#Loadung the model vgg19 that will serve as the base model
model=models.vgg19(pretrained=True).features
#Assigning the GPU to the variable device
device=torch.device( "cuda" if (torch.cuda.is_available()) else 'cpu')
device

device(type='cuda')

# Image Preprocessing

In [21]:
#defing a function that will load the image and perform the required preprocessing and put it on the GPU
def image_loader(path):
    image=Image.open(path)
    #defining the image transformation steps to be performed before feeding them to the model
    loader=transforms.Compose([transforms.Resize((512,512)), transforms.ToTensor()])
    #The preprocessing steps involves resizing the image and then converting it to a tensor
    image=loader(image).unsqueeze(0)
    return image.to(device,torch.float)

#Loading the original and the style image
original_image=image_loader('download.jpeg')
style_image=image_loader('content.jpg')

#Creating the generated image from the original image
generated_image=original_image.clone().requires_grad_(True)

# Feature Representations

In [22]:
#Defining a class that for the model
class VGG(nn.Module):
    def __init__(self):
        super(VGG,self).__init__()
        self.req_features= ['0','5','10','19','28'] 
        #Since we need only the 5 layers in the model so we will be dropping all the rest layers from the features of the model
        self.model=models.vgg19(pretrained=True).features[:29] #model will contain the first 29 layers
    
   
    #x holds the input tensor(image) that will be feeded to each layer
    def forward(self,x):
        #initialize an array that wil hold the activations from the chosen layers
        features=[]
        #Iterate over all the layers of the mode
        for layer_num,layer in enumerate(self.model):
            #activation of the layer will stored in x
            x=layer(x)
            #appending the activation of the selected layers and return the feature array
            if (str(layer_num) in self.req_features):
                features.append(x)
                
        return features

# Content Loss

In [23]:
def calc_content_loss(gen_feat,orig_feat):
    #calculating the content loss of each layer by calculating the MSE between the content and generated features and adding it to content loss
    content_l=torch.mean((gen_feat-orig_feat)**2)
    return content_l

# Style Loss

In [24]:
def calc_style_loss(gen,style):
    #Calculating the gram matrix for the style and the generated image
    batch_size,channel,height,width=gen.shape

    G=torch.mm(gen.view(channel,height*width),gen.view(channel,height*width).t())
    A=torch.mm(style.view(channel,height*width),style.view(channel,height*width).t())
        
    #Calcultating the style loss of each layer by calculating the MSE between the gram matrix of the style image and the generated image and adding it to style loss
    style_l=torch.mean((G-A)**2)
    return style_l

# Total Loss

In [25]:
def calculate_loss(gen_features, orig_feautes, style_featues):
    style_loss=content_loss=0
    for gen,cont,style in zip(gen_features,orig_feautes,style_featues):
        #extracting the dimensions from the generated image
        content_loss+=calc_content_loss(gen,cont)
        style_loss+=calc_style_loss(gen,style)
    
    #calculating the total loss of e th epoch
    total_loss=alpha*content_loss + beta*style_loss 
    return total_loss

In [26]:
# Function to convert tensor to image
def tensor_to_image(tensor):
    image = tensor.clone().detach().cpu()
    image = transforms.ToPILImage()(image.squeeze(0))
    return image

# Training

In [27]:
#Load the model to the GPU
model=VGG().to(device).eval() 

#initialize the paramerters required for fitting the model
epoch=250
lr=0.004
alpha=8
beta=70

#using adam optimizer and it will update the generated image not the model parameter 
optimizer=optim.Adam([generated_image],lr=lr)

#iterating for 1000 times
for e in range (epoch):
    #extracting the features of generated, content and the original required for calculating the loss
    gen_features=model(generated_image)
    orig_feautes=model(original_image)
    style_featues=model(style_image)
    
    #iterating over the activation of each layer and calculate the loss and add it to the content and the style loss
    total_loss=calculate_loss(gen_features, orig_feautes, style_featues)
    #optimize the pixel values of the generated image and backpropagate the loss
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()
    #print the image and save it after each 100 epoch
    if (e + 1) % 10 == 0:
        print(f'Epoch {e + 1}/{epoch}, Total Loss: {total_loss.item()}')
        save_image(generated_image, f"gen_epoch_{e + 1}.png")
        # Convert the tensor to an image and display it
        # img = tensor_to_image(generated_image)
        # plt.imshow(img)
        # plt.axis('off')  # Turn off axis
        # plt.show()


Epoch 10/250, Total Loss: 3140463360.0
Epoch 20/250, Total Loss: 2316333824.0
Epoch 30/250, Total Loss: 1805244032.0
Epoch 40/250, Total Loss: 1513151744.0
Epoch 50/250, Total Loss: 1325280384.0
Epoch 60/250, Total Loss: 1174017792.0
Epoch 70/250, Total Loss: 1045424640.0
Epoch 80/250, Total Loss: 937219520.0
Epoch 90/250, Total Loss: 845201344.0
Epoch 100/250, Total Loss: 766020928.0
Epoch 110/250, Total Loss: 697420416.0
Epoch 120/250, Total Loss: 637348224.0
Epoch 130/250, Total Loss: 584014784.0
Epoch 140/250, Total Loss: 535985920.0
Epoch 150/250, Total Loss: 492142304.0
Epoch 160/250, Total Loss: 451610208.0
Epoch 170/250, Total Loss: 413779296.0
Epoch 180/250, Total Loss: 378271040.0
Epoch 190/250, Total Loss: 344849984.0
Epoch 200/250, Total Loss: 313443296.0
Epoch 210/250, Total Loss: 284046560.0
Epoch 220/250, Total Loss: 256705984.0
Epoch 230/250, Total Loss: 231467008.0
Epoch 240/250, Total Loss: 208393616.0
Epoch 250/250, Total Loss: 187524704.0


In [28]:
save_image(generated_image, "final_generated_image2.png")